<a href="https://colab.research.google.com/github/P0rt/doc2query/blob/main/DOC2QUERY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SET UP
## add the links to gerate the questions for into the list below

## and set the number of questions to generate per document

# New section

In [1]:
topic_links = [
    "https://github.com/javascript-tutorial/en.javascript.info/blob/master/3-frames-and-windows/01-popup-windows/article.md",
    "https://github.com/javascript-tutorial/en.javascript.info/blob/master/5-network/04-fetch-abort/article.md",
    "https://github.com/javascript-tutorial/en.javascript.info/blob/master/5-network/08-xmlhttprequest/article.md",
    "https://github.com/javascript-tutorial/en.javascript.info/blob/master/5-network/11-websocket/article.md"
]

In [2]:
num_questions_per_link = 30


# Generating questions to a page of content in github
1. downloads the contents from a link
2. parses the contents by their headers
3. goes header by header and generates questions for it.
4. writes the questions into a json in format

[
  {
    "url": "...",
    "chapter": "...",
    "questions": [
      "question1",
      "question2",
      "question3"
    ]
  }
]

In [3]:
! pip install transformers -q
! pip install sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 981.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00


In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import requests
import re
import random
random.seed(101010)

### reading github links

In [5]:
def get_md_content(url):
  # Convert the blob url to raw content url
  raw_url = url.replace("github.com", "raw.githubusercontent.com").replace("/blob/", "/")

  response = requests.get(raw_url)
  response.raise_for_status()  # Raise exception for failed requests
  return response.text

def split_md_by_header(md_string):

  sections = re.split(r'(\n?#{1,6} .+)', md_string)
  sections = [section.strip() for section in sections if section.strip()]
  markdown_dict = {}

  # Iterate through sections two at a time (header, content)
  for i in range(0, len(sections)-1, 2):
      header = sections[i]
      content = sections[i+1]
      markdown_dict[header] = content

  return markdown_dict

def get_link_contents(links):
  link_contents = {}
  for link in links:
    link_dict = {}
    content = get_md_content(link)
    link_dict['chapter'] = content # the whole page
    header_dict = split_md_by_header(content)
    link_dict['chapter_headers'] =  header_dict
    link_contents[link] = link_dict

  return link_contents


In [6]:
topic_contents = get_link_contents(topic_links)

### generating questions for headers

In [7]:
model_name = 'doc2query/msmarco-t5-base-v1'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to("cuda")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [8]:
for_json = []

for link in topic_links:
  topic_dict = {}
  print(link)
  print(len(topic_contents[link]['chapter_headers'].keys()))
  topic_dict['link'] = link
  topic_dict['chapteer'] = topic_contents[link]['chapter']
  topic_questions = []
  for header in topic_contents[link]['chapter_headers'].keys():
    text = topic_contents[link]['chapter_headers'][header]
    input_ids = tokenizer.encode(text, max_length=512, truncation=True, return_tensors='pt')
    #print(input_ids.shape)
    outputs = model.generate(
                            input_ids=input_ids.to("cuda"),
                            max_length=128,
                            do_sample=True,
                            top_p=0.95,
                            num_return_sequences=30)

    for i in range(len(outputs)):
      query = tokenizer.decode(outputs[i], skip_special_tokens=True)
      topic_questions.append(query)
  print("There are", len(set(topic_questions)), "questions to sample from")
  questions_to_add = random.sample(list(set(topic_questions)), num_questions_per_link)
  topic_dict['questions'] = {"question"+str(i+1):questions_to_add[i] for i in range(len(questions_to_add))}
  for_json.append(topic_dict)

https://github.com/javascript-tutorial/en.javascript.info/blob/master/3-frames-and-windows/01-popup-windows/article.md
11
There are 319 questions to sample from
https://github.com/javascript-tutorial/en.javascript.info/blob/master/5-network/04-fetch-abort/article.md
5
There are 123 questions to sample from
https://github.com/javascript-tutorial/en.javascript.info/blob/master/5-network/08-xmlhttprequest/article.md
11
There are 313 questions to sample from
https://github.com/javascript-tutorial/en.javascript.info/blob/master/5-network/11-websocket/article.md
10
There are 262 questions to sample from


In [10]:
with open("test.json", "w") as file:
    json.dump(for_json, file)

### sample questions